In [32]:
# Import libraries
import pandas as pd
import numpy as np
import statsmodels.stats.multitest as multi
from collections import Counter
from statsmodels.stats.proportion import proportions_ztest
from scipy import stats
from math import log10
from pyhpo import Ontology
from HPO_functions import *
_ = Ontology()

# -------------------------------------------------------------------

### **Import DECIPHER dataset & process**
- Identify all patients in DECIPHER with a pathogenic/likely pathogenic variant and at least 1 HPO term. 
- Get all propagated HPO IDs and terms for each patient.

In [33]:
# Read DECIPHER csv & filter
decipher_data = pd.read_csv('decipher.csv')
decipher_data = decipher_data[(decipher_data["pathogenicity"] == "Pathogenic") | (decipher_data["pathogenicity"] == "Likely pathogenic")]
decipher_data['hpo_terms_freq'] = decipher_data['phenotype_names'].apply(get_frequency)
decipher_data = decipher_data[decipher_data['phenotype_names'].notna()]

# Get propagated terms
decipher_data["propagated_IDs"] = propagate_HPO_IDs(decipher_data["hpo_accessions"])
decipher_data["propagated_IDs"] = decipher_data["propagated_IDs"].apply('|'.join)
decipher_data['propagated_terms'] = get_HPO_terms(decipher_data['propagated_IDs'])

- Identify top-level HPO terms.

In [34]:
# Get terms directly below "Phenotypic abnormality"
toplevel_IDs = []
for HPO in Ontology.get_hpo_object("Phenotypic abnormality").children:
    toplevel_IDs.append(str(HPO)[0:10])

# Replace "Abnormality of the musculoskeletal system" with abnormalities of the skeletal system, musculature and connective tissue
toplevel_IDs.remove("HP:0033127")
toplevel_IDs.extend(["HP:0000924", "HP:0003549", "HP:0003011"])

# Convert HPO IDs to HPO names
toplevel_terms = []
for HPO in toplevel_IDs:
    toplevel_terms.append(get_HPO_term(HPO))
print(f'{len(toplevel_terms)} Top-Level HPO Terms: {toplevel_terms}')

# Get list of lists of propagated HPO terms for each patient
HPOTerm_list = []
for HPOTerms in decipher_data["propagated_terms"].tolist():
    HPOTerm_list.append(HPOTerms.split('|'))

# Find top-level terms from propagated terms for each patient
HPOTerms_filtered = []
for HPOTerms in HPOTerm_list:
    HPOTerms = [term for term in HPOTerms if term in toplevel_terms] 
    HPOTerms_filtered.append(HPOTerms)
decipher_data["toplevel_terms"] = HPOTerms_filtered

25 Top-Level HPO Terms: ['Abnormality of metabolism/homeostasis', 'Abnormality of prenatal development or birth', 'Abnormality of the eye', 'Neoplasm', 'Abnormality of the thoracic cavity', 'Abnormality of blood and blood-forming tissues', 'Abnormality of the ear', 'Abnormality of the respiratory system', 'Abnormality of the nervous system', 'Abnormality of head or neck', 'Abnormality of the cardiovascular system', 'Abnormality of the digestive system', 'Abnormality of the voice', 'Abnormality of limbs', 'Constitutional symptom', 'Growth abnormality', 'Abnormality of the breast', 'Abnormal cellular phenotype', 'Abnormality of the immune system', 'Abnormality of the genitourinary system', 'Abnormality of the endocrine system', 'Abnormality of the integument', 'Abnormality of the skeletal system', 'Abnormality of connective tissue', 'Abnormality of the musculature']


# -------------------------------------------------------------------

### **Filter DECIPHER using defined gene list**
- Find unique patients with pathogenic or likely pathogenic variants in genes defined in the gene list, who have at least 1 HPO term.

In [35]:
# Filter processed DECIPHER dataset using gene list
gene_list = pd.read_csv('gene_list.csv', header=0)
gene_list = gene_list["gene"].to_list()
decipher_data_filtered = decipher_data[decipher_data["gene"].isin(gene_list)]

# Check which patients have more than 1 variant
duplicates = decipher_data_filtered[decipher_data_filtered.duplicated(subset='# patient_id')]
if len(duplicates) > 0:
    non_unique_patients = list(duplicates['# patient_id'])#.unique())
    print(f"Patients with non-unique variants: {non_unique_patients}\n")
    patient_genes = {}
    for patient_id in non_unique_patients:
        patient_data = decipher_data_filtered[decipher_data_filtered['# patient_id'] == patient_id]
        genes = list(patient_data['gene'].unique())
        patient_genes[patient_id] = genes
    for patient_id, genes in patient_genes.items():
        print(f"Patient {patient_id} has variants in genes: {genes}")
print('\nAll patients with multiple variants have variants within the same gene.')

# Remove duplicate patients
decipher_data_filtered = decipher_data_filtered.drop_duplicates(subset=['# patient_id'])

Patients with non-unique variants: [263014, 264461, 274394, 277208, 293763, 293763, 304945]

Patient 263014 has variants in genes: ['KMT2A']
Patient 264461 has variants in genes: ['KMT2A']
Patient 274394 has variants in genes: ['ACTL6B']
Patient 277208 has variants in genes: ['KMT2A']
Patient 293763 has variants in genes: ['BCOR']
Patient 304945 has variants in genes: ['KMT2A']

All patients with multiple variants have variants within the same gene.


- Find all unique HPO terms across entire DECIPHER dataset.
- Get the frequency and percentage occurance of each term across the gene list-filtered dataset.

In [36]:
# Get all unique HPO terms
allHPOTerms_decipher = set()
decipher_data['propagated_terms'].str.split("|").apply(allHPOTerms_decipher.update)
allHPOTerms_decipher = list(allHPOTerms_decipher)
freqHPOTerms_decipher = len(allHPOTerms_decipher)

# Find patient frequency
patient_freq = len(decipher_data_filtered)

# Go through list of strings, to create list of lists
HPOTerms_list = decipher_data_filtered["propagated_terms"].tolist()
HPOTerms_longlist = []
for HPOTerms in HPOTerms_list:
    HPOTerms = HPOTerms.split('|')
    HPOTerms_longlist.append(HPOTerms)

# Count number of terms across patients 
HPOTerms_longlist_flat = [item for sublist in HPOTerms_longlist for item in sublist]
HPOTerms_freq = Counter(HPOTerms_longlist_flat)

# Identify number of terms 
HPOTerms_freqAll = {}
for term in allHPOTerms_decipher:
    if term in HPOTerms_freq:
        HPOTerms_freqAll[term] = HPOTerms_freq[term]
    else:
        HPOTerms_freqAll[term] = 0

# Identify percentage of terms 
HPOTerms_percAll = {}
for term, freq in HPOTerms_freqAll.items():
    term = str(Ontology.get_hpo_object(term))
    percent = (freq/patient_freq)*100
    HPOTerms_percAll[term[13:]] = round(percent, 2)

# -------------------------------------------------------------------

### **Reverse filter DECIPHER using defined gene list**
- Find unique patients with pathogenic or likely pathogenic variants in genes that are not in the defined gene list, who have at least 1 HPO term.

In [37]:
# Filter processed DECIPHER dataset using gene list
decipher_data_rev = decipher_data[~decipher_data["gene"].isin(gene_list)]
decipher_data_rev = decipher_data_rev.drop_duplicates(subset=['# patient_id'])

# Create list of HPO terms for each patient
rev_HPOTerms_list = decipher_data_rev["propagated_terms"].tolist()
rev_patient_freq = len(decipher_data_rev)

# Go through list of strings, to create list of lists
rev_HPOTerms_longlist = []
for HPOTerms in rev_HPOTerms_list:
    HPOTerms = HPOTerms.split('|')
    rev_HPOTerms_longlist.append(HPOTerms)

# Count number of terms across patients 
rev_HPOTerms_longlist_flat = [item for sublist in rev_HPOTerms_longlist for item in sublist]
rev_HPOTerms_freq = Counter(rev_HPOTerms_longlist_flat)

# Identify number of terms 
rev_HPOTerms_freqAll = {}
for term in allHPOTerms_decipher:
    if term in rev_HPOTerms_freq:
        rev_HPOTerms_freqAll[term] = rev_HPOTerms_freq[term]
    else:
        rev_HPOTerms_freqAll[term] = 0

# Identify percentage of terms 
rev_HPOTerms_percAll = {}
for term, freq in rev_HPOTerms_freqAll.items():
    term = str(Ontology.get_hpo_object(term))
    percent = (freq/rev_patient_freq)*100
    rev_HPOTerms_percAll[term[13:]] = round(percent, 2)

# -------------------------------------------------------------------

### **Compare occurance of HPO terms between the two identified groups**
- Gather frequencies and percentages for both groups.

In [38]:
genelist_rev_HPO = pd.DataFrame({'HPO_term': list(HPOTerms_percAll.keys()), 'genelist_percent': list(HPOTerms_percAll.values()), 'genelist_freq': list(HPOTerms_freqAll.values()), 
'rev_percent': list(rev_HPOTerms_percAll.values()), 'rev_freq': list(rev_HPOTerms_freqAll.values())})
genelist_rev_HPO["total_freq"] = genelist_rev_HPO["genelist_freq"] + genelist_rev_HPO["rev_freq"] 
genelist_rev_HPO = genelist_rev_HPO.sort_values('HPO_term')

- Using a two-proportions z-test and FDR (BH) correction from statsmodels, compare the proportion of patients in each group with each phenotype, to identify significantly increased/decreased phenotypes.

In [39]:
# Loop through each HPO term and get p-value
for index, row in genelist_rev_HPO.iterrows():
    HPO_term = row["HPO_term"]
    genelist_freq = row["genelist_freq"]
    rev_freq = row["rev_freq"]
    frequencies = np.array([genelist_freq, rev_freq])
    totals = np.array([patient_freq, rev_patient_freq])
    stat, p_value = proportions_ztest(count=frequencies, nobs=totals, alternative="two-sided")
    genelist_rev_HPO.loc[index, "p_value"] = p_value

# Adjust p-value (BH) and identify significantly different terms at p<0.005
p_values = np.array(genelist_rev_HPO["p_value"])
p_values[np.isnan(p_values)] = 1
p_adj = multi.fdrcorrection(p_values, alpha=0.05, method='indep')
genelist_rev_HPO["adj_p_value"] = p_adj[1]
genelist_rev_HPO['significant'] = np.where(genelist_rev_HPO['adj_p_value']<0.005, "Y", "N")

# Get significant HPO terms that are present in at least 5% of patients in the gene-list filtered group
genelist_rev_HPO_significant = genelist_rev_HPO[(genelist_rev_HPO['significant']=="Y") & (genelist_rev_HPO['genelist_percent']>5)]
genelist_rev_HPO_significant['change'] = np.where(genelist_rev_HPO_significant['genelist_percent']>genelist_rev_HPO_significant['rev_percent'], "Increase", "Decrease")

/Users/alicesmail/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
/Users/alicesmail/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
/var/folders/hw/lt7xkjh55hz1_s2z24sn1srh0000gn/T/ipykernel_34347/1789315365.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genelist_rev_HPO_significant['change'] = np.where(genelist_rev_HPO_significant['genelist_percent']>genelist_rev_HPO_significant['rev_percent'], "Increase", "Decrease")


In [40]:
# Add HPO IDs
ids = []
for term in genelist_rev_HPO['HPO_term']:
    id = get_HPO_id(term)
    ids.append(id)
genelist_rev_HPO['HPO_id'] = ids

# Find percentage difference and log10 pvalue
genelist_rev_HPO['delta'] = genelist_rev_HPO['genelist_percent'] - genelist_rev_HPO['rev_percent']
log10s = []
for pval in genelist_rev_HPO['adj_p_value']:
    log10s.append(-1*(log10(pval)))
genelist_rev_HPO['-log10padj'] = log10s

# Find top-level terms
genelist_rev_HPO_toplevel = genelist_rev_HPO[genelist_rev_HPO['HPO_term'].isin(toplevel_terms)]
display(genelist_rev_HPO_toplevel)

,HPO_term,genelist_percent,genelist_freq,rev_percent,rev_freq,total_freq,p_value,adj_p_value,significant,HPO_id,delta,-log10padj
769,Abnormal cellular phenotype,0.00,0,0.45,23,23,1.468560e-01,8.163002e-01,N,HP:0025354,-0.45,0.088150
2545,Abnormality of blood and blood-forming tissues,1.30,6,2.07,105,111,2.570789e-01,8.163002e-01,N,HP:0001871,-0.77,0.088150
2760,Abnormality of connective tissue,6.49,30,7.02,356,386,6.696659e-01,8.163002e-01,N,HP:0003549,-0.53,0.088150
2541,Abnormality of head or neck,76.19,352,62.82,3185,3537,1.008010e-08,1.333825e-06,Y,HP:0000152,13.37,5.874901
960,Abnormality of limbs,39.83,184,30.28,1535,1719,2.171448e-05,1.238733e-03,Y,HP:0040064,9.55,2.907022
565,Abnormality of metabolism/homeostasis,4.98,23,4.32,219,242,5.074388e-01,8.163002e-01,N,HP:0001939,0.66,0.088150
822,Abnormality of prenatal development or birth,3.90,18,4.58,232,250,5.006672e-01,8.163002e-01,N,HP:0001197,-0.68,0.088150
2464,Abnormality of the breast,3.25,15,3.35,170,185,9.031624e-01,9.282816e-01,N,HP:0000769,-0.10,0.032320
3773,Abnormality of the cardiovascular system,14.94,69,14.26,723,792,6.918115e-01,8.163002e-01,N,HP:0001626,0.68,0.088150
3845,Abnormality of the digestive system,26.84,124,16.67,845,969,3.644819e-08,4.040824e-06,Y,HP:0025031,10.17,5.393530


# -------------------------------------------------------------------

### **Compare frequency of HPO terms between the two identified groups**
- Identify and compare number of top-level HPO terms per patient between the two groups.

In [41]:
# Count top-level HPO terms for each gene list patient
genelist_toplevel = decipher_data_filtered[["# patient_id", "toplevel_terms"]]
genelist_toplevel["toplevel_terms"] = genelist_toplevel["toplevel_terms"].apply('|'.join)
genelist_toplevel['HPO_toplevel_terms_freq'] = genelist_toplevel['toplevel_terms'].apply(get_frequency)

# Count top-level HPO terms for each patient in the rest of DECIPHER
rev_toplevel = decipher_data_rev[["# patient_id", "toplevel_terms"]]
rev_toplevel["toplevel_terms"] = rev_toplevel["toplevel_terms"].apply('|'.join)
rev_toplevel['HPO_toplevel_terms_freq'] = rev_toplevel['toplevel_terms'].apply(get_frequency)

# Find number of patients with each HPO term frequency 
genelist_counts_toplevel = dict(Counter(genelist_toplevel['HPO_toplevel_terms_freq']))
rev_counts_toplevel = dict(Counter(rev_toplevel['HPO_toplevel_terms_freq']))

# Calculate percent of patients in each group with each frequency
genelist_percent_toplevel = {key: (value / patient_freq)*100 for key, value in genelist_counts_toplevel.items()}
rev_percent_toplevel = {key: (value / rev_patient_freq)*100 for key, value in rev_counts_toplevel.items()}

# Create dataframe comparing gene list patients with patients in the rest of DECIPHER
toplevel_hpo_percent = pd.DataFrame({"gene_list_percent": pd.Series(genelist_percent_toplevel), "rev_percent": pd.Series(rev_percent_toplevel)})
display(toplevel_hpo_percent)

# Get array of frequencies for patients in each group
rev_toplevel_arr = pd.array(rev_toplevel['HPO_toplevel_terms_freq'])
genelist_toplevel_arr = pd.array(genelist_toplevel['HPO_toplevel_terms_freq'])

# Summary statistics
rev_stats = (pd.DataFrame(rev_toplevel_arr).agg(["count", "min", "max", "median", "mean", "skew"]).rename(columns={pd.DataFrame(rev_toplevel_arr).columns[0]: 'rev_list'}).round(decimals=2))
genelist_stats = pd.DataFrame(genelist_toplevel_arr).agg(["count", "min", "max", "median", "mean", "skew"]).rename(columns={pd.DataFrame(genelist_toplevel_arr).columns[0]: 'gene_list'}).round(decimals=2)
display(pd.concat([rev_stats, genelist_stats], axis=1))

# Mann Whitney U test
mann_whitney_toplevel = stats.mannwhitneyu(genelist_toplevel_arr, rev_toplevel_arr, alternative='two-sided')
print(f'\nP-value: {mann_whitney_toplevel.pvalue}')

/var/folders/hw/lt7xkjh55hz1_s2z24sn1srh0000gn/T/ipykernel_34347/2485444733.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genelist_toplevel["toplevel_terms"] = genelist_toplevel["toplevel_terms"].apply('|'.join)
/var/folders/hw/lt7xkjh55hz1_s2z24sn1srh0000gn/T/ipykernel_34347/2485444733.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genelist_toplevel['HPO_toplevel_terms_freq'] = genelist_toplevel['toplevel_terms'].apply(get_frequency)
/var/folders/hw/lt7xkjh55hz1_s2z24sn1srh0000gn/T/ipykernel_343

,gene_list_percent,rev_percent
1,7.359307,14.911243
2,7.359307,9.072978
3,7.142857,13.708087
4,13.203463,15.088757
5,17.748918,14.280079
6,16.233766,11.775148
7,13.203463,9.112426
8,8.225108,5.147929
9,4.545455,3.136095
10,2.597403,2.169625


,rev_list,gene_list
count,5070.00,462.00
min,1.00,1.00
max,18.00,14.00
median,4.00,5.00
mean,4.50,5.36
skew,0.55,0.26



P-value: 1.513148080718329e-13


- Identify and compare number of unpropagated terms per patient between the two groups.

In [42]:
# Find number of patients with each HPO term frequency 
genelist_counts_unpropagated = dict(Counter(decipher_data_filtered['hpo_terms_freq']))
rev_counts_unpropagated = dict(Counter(decipher_data_rev['hpo_terms_freq']))

# Calculate percent of patients in each group with each frequency
gene_list_percent_unpropagated = {key: (value / patient_freq)*100 for key, value in genelist_counts_unpropagated.items()}
rev_percent_unpropagated = {key: (value / rev_patient_freq)*100 for key, value in rev_counts_unpropagated.items()}

# Create percentage dataframe comparing gene list patients with patients in the rest of DECIPHER
unpropagated_hpo_percent = pd.DataFrame({"genelist_percent": pd.Series(gene_list_percent_unpropagated), "rev_percent": pd.Series(rev_percent_unpropagated)})
unpropagated_hpo_percent = unpropagated_hpo_percent.fillna(0)

# Create frequency dataframe comparing gene list patients with patients in the rest of DECIPHER
unpropagated_hpo_freq = pd.DataFrame({"genelist_freq": pd.Series(genelist_counts_unpropagated), "rev_freq": pd.Series(rev_counts_unpropagated)})
unpropagated_hpo_freq = unpropagated_hpo_freq.fillna(0)

# Convert to histogram format
histogram = pd.DataFrame(columns = ["bin", "genelist_percent", "rev_percent"])
histogram["bin"] = ("1-4", "5-9", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39")
histogram["genelist_percent"] = (sum(unpropagated_hpo_percent["genelist_percent"][0:4]), sum(unpropagated_hpo_percent["genelist_percent"][4:9]), sum(unpropagated_hpo_percent["genelist_percent"][9:14]), sum(unpropagated_hpo_percent["genelist_percent"][14:19]), sum(unpropagated_hpo_percent["genelist_percent"][19:24]), sum(unpropagated_hpo_percent["genelist_percent"][24:29]), sum(unpropagated_hpo_percent["genelist_percent"][29:34]), sum(unpropagated_hpo_percent["genelist_percent"][34:39]))
histogram["rev_percent"] = (sum(unpropagated_hpo_percent["rev_percent"][0:4]), sum(unpropagated_hpo_percent["rev_percent"][4:9]), sum(unpropagated_hpo_percent["rev_percent"][9:14]), sum(unpropagated_hpo_percent["rev_percent"][14:19]), sum(unpropagated_hpo_percent["rev_percent"][19:24]), sum(unpropagated_hpo_percent["rev_percent"][24:29]), sum(unpropagated_hpo_percent["rev_percent"][29:34]), sum(unpropagated_hpo_percent["rev_percent"][34:39]))
display(histogram)

# Get array of frequencies for patients in each group
rev_unprop_arr = np.array(decipher_data_rev['hpo_terms_freq'])
genelist_unprop_arr = np.array(decipher_data_filtered['hpo_terms_freq'])

# Summary statistics
rev_stats = (pd.DataFrame(rev_unprop_arr).agg(["count", "min", "max", "median", "mean", "skew"]).rename(columns={pd.DataFrame(rev_unprop_arr).columns[0]: 'rev_list'}).round(decimals=2))
genelist_stats = (pd.DataFrame(genelist_unprop_arr).agg(["count", "min", "max", "median", "mean", "skew"]).rename(columns={pd.DataFrame(genelist_unprop_arr).columns[0]: 'gene_list'}).round(decimals=2))
display(pd.concat([rev_stats, genelist_stats], axis=1))

# Mann-Whitney U test
mann_whitney_unprop = stats.mannwhitneyu(genelist_unprop_arr, rev_unprop_arr, alternative='two-sided')
print(f'\nP-value: {mann_whitney_unprop.pvalue}')

,bin,genelist_percent,rev_percent
0,1-4,22.727273,36.114398
1,5-9,45.887446,42.781065
2,10-14,23.160173,15.897436
3,15-19,5.194805,3.786982
4,20-24,1.731602,1.084813
5,25-29,0.649351,0.216963
6,30-34,0.649351,0.118343
7,35-39,0.000000,0.000000


,rev_list,gene_list
count,5070.00,462.00
min,1.00,1.00
max,36.00,39.00
median,6.00,7.00
mean,6.63,8.01
skew,1.36,1.71



P-value: 1.5116338173521496e-10


# -------------------------------------------------------------------

In [43]:
decipher_data_filtered.to_csv('decipher_data_filtered.csv')